In [1]:
import pandas as pd
import dill
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import recall_score, precision_score, roc_auc_score, roc_curve #,scorer
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
#working with text
from sklearn.feature_extraction.text import TfidfVectorizer
#normalizing data
from sklearn.preprocessing import StandardScaler
#pipeline
from sklearn.pipeline import Pipeline, FeatureUnion
#imputer
from sklearn.impute import SimpleImputer

dill._dill._reverse_typemap['ClassType'] = type
import sklearn.datasets

Загрузим данные

In [4]:
df = pd.read_csv("labeled.csv")
df.head(3)

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14412 entries, 0 to 14411
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   comment  14412 non-null  object 
 1   toxic    14412 non-null  float64
dtypes: float64(1), object(1)
memory usage: 225.3+ KB


In [6]:
df.describe()

,toxic
count,14412.000000
mean,0.334860
std,0.471958
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [7]:
df['toxic'].value_counts()

0.0    9586
1.0    4826
Name: toxic, dtype: int64

Разделим данные на train/test и сохраним тестовую выборку на диск (здесь мы ее касаться уже не будем)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df, df['toxic'], test_size=0.33, random_state=42)
#save test
X_test.to_csv("X_test.csv", index=None)
y_test.to_csv("y_test.csv", index=None)
#save train
X_train.to_csv("X_train.csv", index=None)
y_train.to_csv("y_train.csv", index=None)

In [9]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X[self.key] = X[self.key].fillna(self.value)
        return X

In [10]:
features = 'comment'
target = 'toxic'

In [11]:
#combine
comment = Pipeline([
                ('imputer', TextImputer('comment', '')),
                ('selector', ColumnSelector(key='comment')),
                ('tfidf', TfidfVectorizer(max_df=0.9, min_df=10))
            ])


feats = FeatureUnion([('comment', comment)])

In [12]:
%%time

pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier()),
])

pipeline.fit(X_train, y_train)

CPU times: user 4.61 s, sys: 11.5 ms, total: 4.62 s
Wall time: 4.64 s


In [13]:
pipeline.steps

[('features', FeatureUnion(transformer_list=[('comment',
                                  Pipeline(steps=[('imputer',
                                                   TextImputer(key='comment',
                                                               value='')),
                                                  ('selector',
                                                   ColumnSelector(key='comment')),
                                                  ('tfidf',
                                                   TfidfVectorizer(max_df=0.9,
                                                                   min_df=10))]))])),
 ('classifier', GradientBoostingClassifier())]

Сохраним модель (пайплайн)

In [14]:
with open("gbc_pipeline.dill", "wb") as f:
    dill.dump(pipeline, f)